<a href="https://colab.research.google.com/github/M4ri4Edd/resumoNoticias/blob/main/ResumirNoticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets sacremoses beautifulsoup4 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatib

# Bibliotecas

In [ ]:
import pandas as pd
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import Dataset
import requests
from bs4 import BeautifulSoup

# Carregar e Pré-processar o Dataset

In [ ]:
df = pd.read_csv("/content/articles_menor_limpo.csv", delimiter=",", quotechar='"', encoding="utf-8", on_bad_lines="skip", low_memory=False)

# Selecionar apenas as colunas relevantes
df = df[['text', 'title']]  # Ajuste os nomes conforme necessário

# Remover valores nulos
df = df.dropna()

In [ ]:
dataset = Dataset.from_pandas(df)

# Preparar o Modelo BART-pt

In [ ]:
modelo = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(modelo)
model = BartForConditionalGeneration.from_pretrained(modelo)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Tokenize os dados:

In [ ]:
def tokenize_function(examples):
    inputs = tokenizer(examples["text"], max_length=1024, truncation=True, padding="max_length")
    targets = tokenizer(examples["title"], max_length=128, truncation=True, padding="max_length")
    return {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"], "labels": targets["input_ids"]}

dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7942 [00:00<?, ? examples/s]

# Treinar o Modelo

In [ ]:
from transformers import TrainingArguments, Trainer
#não recomendo mexer aqui, o loss do treinamento e validação está a uma taxa ótima desse jeito
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
)
#ainda é possível avaliar se mudanças aqui fazem alguma diferença
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset.shuffle().select(range(5000)),  # Amostra para treino
    eval_dataset=dataset.shuffle().select(range(500, 1000)),  # Amostra para validação
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: luzdudinha (luzdudinha-ifpi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


# Criar Função para Resumir Notícias a partir de um Link

In [ ]:
def get_news_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    paragraphs = soup.find_all('p')
    news_text = " ".join([p.get_text() for p in paragraphs])

    return news_text

import torch

def summarize_news(url):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Obtém o conteúdo da notícia
    news_text = get_news_content(url)

    # Move o modelo para o dispositivo correto
    model.to(device)

    # Tokeniza o texto e move os inputs para o mesmo dispositivo do modelo
    inputs = tokenizer(
        news_text,
        return_tensors="pt",
        max_length=1024,  # Pode aumentar esse valor para suportar textos maiores
        truncation=True,
        padding="longest"  # Garante que todas as entradas tenham o mesmo tamanho
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move os tensores

    # Gera o resumo
    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=60, length_penalty=1.5, num_beams=8)
    #até agora, esses números geraram os melhores resultados

    # Decodifica o resultado
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return summary

# Testar com um link
url = "https://www.terra.com.br/noticias/deputada-quer-diminuir-influencia-politica-no-ibge-presidente-e-diretores-deverao-passar-por-crivo-do-senado,afd053ba225103fc0c8190b7af998015z1y8rbth.html"
print(summarize_news(url))


## Grupo pra testes:

* [Resgate de jaguatirica](https://www.cacador.net/noticias/geral/2025/03/14/boa-noticia-jaguatirica-resgatada-em-cacador-e-devolvida-a-natureza-62887)

* [Crise climática](https://www.bbc.com/portuguese/articles/cvg9vj823xeo)

* [Joana da Paz](https://www.ingresso.com/noticias/quem-foi-joana-da-paz-mulher-inspirou-filme-vitoria-estrelado-fernanda-montenegro)

* [Supercopa feminina](https://www.terra.com.br/esportes/futebol/futebol-feminino/sao-paulo-e-corinthians-decidem-a-supercopa-feminina-assista-ao-vivo,44596136903496a378a9b82cd739d506tfjrndny.html)

* [DJ da festa do Neymar](https://www.diariodocentrodomundo.com.br/essencial/falta-de-respeito-o-que-diz-dj-da-festa-de-neymar-apos-farra-em-sitio-de-sp/)

* [Influência política no IBGE](https://www.terra.com.br/noticias/deputada-quer-diminuir-influencia-politica-no-ibge-presidente-e-diretores-deverao-passar-por-crivo-do-senado,afd053ba225103fc0c8190b7af998015z1y8rbth.html)